In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import xgboost as xgb
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
import datetime
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('data/cleaned.csv.gz')
data

,id,text,create_at,geo,retweet_count,like_count,hashtags,username,following,followers,user_total_tweets,user_likes_count,cleaned_text
0,1.59122e+18,'Like a 1930s Dictator': Vitalik Buterin Lays ...,2022-11-11 23:59:59+00:00,"For now, Earth",0,1,#ftx,johnmorganFL,630.0,28492,240872.0,111,like dictator vitalik buterin lay sam bankmanf...
1,1.59122e+18,"💸 $1,000 #Giveaway 💸\n\nWe'll select 10 winner...",2022-11-11 23:59:57+00:00,Panama,8866,0,#ftx,KuKzNFT,594.0,1014,4452.0,8272,giveaway select winner randomly award account...
2,1.59122e+18,#FTX crazy Shit! #Crypto https://t.co/2GO4X3u2pt,2022-11-11 23:59:55+00:00,"Seattle, WA",0,0,#ftx,MeAndMySports,315.0,342,3032.0,1572,ftx crazy shit crypto
3,1.59122e+18,"Yo @SBF_FTX, draw me like one of your French g...",2022-11-11 23:59:53+00:00,"London, England",0,0,#ftx,thedoc7er,1513.0,296,4541.0,3626,draw like french girl ftxcrash ftx binance czb...
4,1.59122e+18,@BitcoinMagazine you you simpletons cant under...,2022-11-11 23:59:51+00:00,NaN,0,1,#ftx,jesseypaul4,123.0,44,317.0,81,simpleton understand crypto wallet like exodus...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
263494,1.59156e+18,"🔴 WAR IN 🇺🇦: FRANCE 24 reporter @cntrentF24, w...",2022-11-12 22:52:11+00:00,Hauts de France 🇫🇷 🇪🇺,20,0,#ukraine,annetteashley61,10522.0,9603,437088.0,291392,war france reporter spend lot time report ukra...
263495,1.59156e+18,People still celebrating in Kherson and I love...,2022-11-12 22:52:09+00:00,"Belgrade, Serbia, Europe",1041,0,#ukraine,svrhovac,222.0,150,9174.0,16675,people celebrate kherson love bit ukraine kherson
263496,1.59156e+18,A remarkable story #Ukraine https://t.co/eeBLc...,2022-11-12 22:52:04+00:00,Guernsey,0,0,#ukraine,TimBarker10,2668.0,1217,3290.0,5927,remarkable story ukraine
263497,1.59156e+18,#Ukraine: Relief in liberated #Kherson after 8...,2022-11-12 22:52:02+00:00,🇮🇪🇪🇺🇺🇦🎮,189,0,#ukraine,Aontaithe2021,5001.0,2376,392315.0,308109,ukraine relief liberated kherson month putin ...


In [3]:
data.columns

Index(['id', 'text', 'create_at', 'geo', 'retweet_count', 'like_count',
       'hashtags', 'username', 'following', 'followers', 'user_total_tweets',
       'user_likes_count', 'cleaned_text'],
      dtype='object')

In [4]:
data['hashtags'].value_counts()

#ukraine          27000
#covid            27000
#christmas        26690
#vegan            26651
#climatechange    26089
#ftx              24076
22.0                  1
ShopParaTi            1
Name: hashtags, dtype: int64

remove tags

In [5]:
data = data[(data['hashtags']!='ShopParaTi') & (data['hashtags']!='22.0')]
data['hashtags'].value_counts()

#ukraine          27000
#covid            27000
#christmas        26690
#vegan            26651
#climatechange    26089
#ftx              24076
Name: hashtags, dtype: int64

one-hot encoding hashtags

In [6]:
data = pd.get_dummies(data=data, columns=['hashtags'])
data.columns

Index(['id', 'text', 'create_at', 'geo', 'retweet_count', 'like_count',
       'username', 'following', 'followers', 'user_total_tweets',
       'user_likes_count', 'cleaned_text', 'hashtags_#christmas',
       'hashtags_#climatechange', 'hashtags_#covid', 'hashtags_#ftx',
       'hashtags_#ukraine', 'hashtags_#vegan'],
      dtype='object')

get useful columns

In [7]:
useful_columns = [
    'following',
    'followers', 
    'user_total_tweets',
    'user_likes_count',
    'hashtags_#christmas', 
    'hashtags_#climatechange',
    'hashtags_#covid', 
    'hashtags_#ftx', 
    'hashtags_#ukraine',
    'hashtags_#vegan',
    'retweet_count', 
    'like_count'
]

all_features = [
    'following',
    'followers', 
    'user_total_tweets',
    'user_likes_count', 
    'hashtags_#christmas', 
    'hashtags_#climatechange',
    'hashtags_#covid', 
    'hashtags_#ftx', 
    'hashtags_#ukraine',
    'hashtags_#vegan'
]
data_baseline = data[useful_columns]

remove NaN and transform data type

In [8]:
data_baseline.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 263497 entries, 0 to 263498
Data columns (total 12 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   following                157506 non-null  float64
 1   followers                157506 non-null  object 
 2   user_total_tweets        157506 non-null  float64
 3   user_likes_count         157506 non-null  object 
 4   hashtags_#christmas      263497 non-null  uint8  
 5   hashtags_#climatechange  263497 non-null  uint8  
 6   hashtags_#covid          263497 non-null  uint8  
 7   hashtags_#ftx            263497 non-null  uint8  
 8   hashtags_#ukraine        263497 non-null  uint8  
 9   hashtags_#vegan          263497 non-null  uint8  
 10  retweet_count            157506 non-null  object 
 11  like_count               157506 non-null  object 
dtypes: float64(2), object(4), uint8(6)
memory usage: 15.6+ MB


In [9]:
data_baseline.dropna(inplace=True)
data_baseline[['retweet_count','like_count','following','followers','user_total_tweets','user_likes_count']] = data_baseline[['retweet_count','like_count','following','followers','user_total_tweets','user_likes_count']].astype('float')

In [10]:
data_baseline.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157506 entries, 0 to 263498
Data columns (total 12 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   following                157506 non-null  float64
 1   followers                157506 non-null  float64
 2   user_total_tweets        157506 non-null  float64
 3   user_likes_count         157506 non-null  float64
 4   hashtags_#christmas      157506 non-null  uint8  
 5   hashtags_#climatechange  157506 non-null  uint8  
 6   hashtags_#covid          157506 non-null  uint8  
 7   hashtags_#ftx            157506 non-null  uint8  
 8   hashtags_#ukraine        157506 non-null  uint8  
 9   hashtags_#vegan          157506 non-null  uint8  
 10  retweet_count            157506 non-null  float64
 11  like_count               157506 non-null  float64
dtypes: float64(6), uint8(6)
memory usage: 9.3 MB


put data into bins

baseline (dummy)

In [11]:
features = data_baseline[all_features]
features.head(1)

,following,followers,user_total_tweets,user_likes_count,hashtags_#christmas,hashtags_#climatechange,hashtags_#covid,hashtags_#ftx,hashtags_#ukraine,hashtags_#vegan
0,630.0,28492.0,240872.0,111.0,0,0,0,1,0,0


In [12]:
labels = data_baseline['like_count']
labels

0         1.0
1         0.0
2         0.0
3         0.0
4         1.0
         ... 
263494    0.0
263495    0.0
263496    0.0
263497    0.0
263498    0.0
Name: like_count, Length: 157506, dtype: float64

In [13]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [14]:
len(X_test), len(y_test)

(31502, 31502)

In [15]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [16]:
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_log_error

dummy1 = DummyRegressor()
dummy1.fit(X_train_scaled, y_train)
mean_squared_log_error(y_test, dummy1.predict(X_test_scaled))

1.29586170440988

In [17]:
dummy2 = DummyRegressor(strategy='constant', constant=0)
dummy2.fit(X_train_scaled, y_train)
mean_squared_log_error(y_test, dummy2.predict(X_test_scaled))

0.4255747839075408

baseline 2 (selected features)

In [18]:
rf = RandomForestRegressor(random_state=0)
rf.fit(X_train_scaled,y_train)
mean_squared_log_error(y_test, rf.predict(X_test_scaled))

0.548265166813657

In [19]:
ada_reg = AdaBoostRegressor(random_state=0, n_estimators = 50)
ada_reg.fit(X_train_scaled,y_train)
mean_squared_log_error(y_test, ada_reg.predict(X_test_scaled))

2.87016378588174

In [20]:
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train_scaled, y_train)
mean_squared_log_error(y_test, knn.predict(X_test_scaled))

0.4988386350101342